Consultar o valor do Dolar por dia com API

In [1]:
import pandas as pd
import requests
from datetime import date

In [6]:

ini = "04-01-2025"
fim = date.today().strftime("%m-%d-%Y")
url = (
  "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
  f"CotacaoDolarPeriodo(dataInicial=@ini,dataFinalCotacao=@fim)?"
  f"@ini='{ini}'&@fim='{fim}'&$select=cotacaoCompra,dataHoraCotacao&$top=10000&$format=json"
)

data = requests.get(url, timeout=30).json()["value"]
df = pd.DataFrame(data)
df["data"] = pd.to_datetime(df["dataHoraCotacao"]).dt.date

# última cotação do dia (fechamento)
df_daily = df.sort_values("dataHoraCotacao").groupby("data").tail(1)
df_daily = df_daily[["data","cotacaoCompra"]].reset_index(drop=True)

# formata data para dd/mm/yyyy
df_daily["data"] = pd.to_datetime(df_daily["data"]).dt.strftime("%d/%m/%Y")
df_daily

,data,cotacaoCompra
0,01/04/2025,5.7045
1,02/04/2025,5.6918
2,03/04/2025,5.6062
3,04/04/2025,5.7771
4,07/04/2025,5.8871
...,...,...
140,20/10/2025,5.3765
141,21/10/2025,5.3842
142,22/10/2025,5.3892
143,23/10/2025,5.3834


In [ ]:
import duckdb

DB_PATH = "dados_dolar.duckdb"
con = duckdb.connect(DB_PATH)

# 1) cria tabela (se não existir)
con.execute("""
CREATE TABLE IF NOT EXISTS dolar_diario (
  data DATE,
  cotacao_compra DOUBLE
);
""")

# 2) registra o DataFrame atual como uma view temporária
con.register("df_stage", df_daily)

# 3) upsert por data (converte 'dd/mm/yyyy' -> DATE)
con.execute("""
MERGE INTO dolar_diario AS t
USING (
  SELECT
    STRPTIME(data, '%d/%m/%Y')::DATE AS data,
    CAST(cotacaoCompra AS DOUBLE)     AS cotacao_compra
  FROM df_stage
) AS s
ON t.data = s.data
WHEN MATCHED THEN UPDATE SET
  cotacao_compra = s.cotacao_compra
WHEN NOT MATCHED THEN INSERT (data, cotacao_compra)
VALUES (s.data, s.cotacao_compra);
""")

# 4) checagem rápida
print(con.execute("""
SELECT *
FROM dolar_diario
ORDER BY data DESC
LIMIT 10;
""").fetchdf())

con.execute("""
COPY dolar_diario TO './exports/dolar_diario.csv' (HEADER, DELIMITER ',');
""")

con.close()


In [ ]:
!pip install yfinance

In [ ]:
# pip install yfinance duckdb pandas
import yfinance as yf
from datetime import date # A 'date' provavelmente foi importada no código original

# ================================
# Configurações
# ================================
DB_PATH = "dados_dolar.duckdb"
TABELA = "dolar_ohlc"
ANO = date.today().year

inicio = pd.Timestamp(ANO, 4, 1)
fim_desejado = pd.Timestamp(ANO, 10, 31)
fim = min(fim_desejado, pd.Timestamp.today().normalize())

ticker = "USDBRL=X"  # dólar/real no Yahoo Finance

# ================================
# 1) Baixar OHLC (mercado) do Yahoo
# ================================
df = yf.download(
  ticker,
  start=inicio,
  end=fim + pd.Timedelta(days=1), # inclui o último dia
  interval="1d",
  auto_adjust=False,
  progress=False,
)

if df.empty:
  raise SystemExit("Nenhum dado retornado pelo Yahoo Finance para o período.")

ohlc = (
  df.reset_index()[["Date", "Open", "High", "Low", "Close"]]
   .rename(columns={
     "Date": "data",
     "Open": "abertura",
     "High": "alta",
     "Low": "baixa",
     "Close": "fechamento",
   })
)

# Arredonda (opcional)
ohlc[["abertura","fechamento","alta","baixa"]] = (
  ohlc[["abertura","fechamento","alta","baixa"]].round(4)
)

print("Prévia OHLC:")
print(ohlc.head())

# ================================
# 2) Salvar no DuckDB (UPSERT simples)
#  - usamos INSERT OR REPLACE com PK(data)
#  - carregamos o DF numa tabela temp 'stage' para evitar BinderException
# ================================
con = duckdb.connect(DB_PATH)

# cria tabela se não existir
con.execute(f"""
CREATE TABLE IF NOT EXISTS {TABELA} (
 data DATE PRIMARY KEY,
 abertura  DOUBLE,
 fechamento DOUBLE,
 alta    DOUBLE,
 baixa   DOUBLE
);
""")

# registra o DF e materializa uma tabela temporária
con.register("df_stage", ohlc)
con.execute("CREATE OR REPLACE TEMP TABLE stage AS SELECT * FROM df_stage;")


preview = con.execute(f"""
SELECT * FROM {TABELA}
ORDER BY data DESC
LIMIT 10;
""").fetchdf()

con.execute("""
COPY dolar_ohlc TO './exports/dolar_ohlc.csv' (HEADER, DELIMITER ',');
""")

con.close()

# exportar ohlc to csv
ohlc.to_csv('./exports/dolar_ohlc_yahoo.csv', index=False)

Prévia OHLC:
Price        data abertura     alta    baixa fechamento
Ticker            USDBRL=X USDBRL=X USDBRL=X   USDBRL=X
0      2025-04-01   5.6984   5.7484   5.6815     5.6984
1      2025-04-02   5.6813   5.7120   5.6583     5.6813
2      2025-04-03   5.6637   5.7476   5.5867     5.6637
3      2025-04-04   5.6306   5.8221   5.6199     5.6306
4      2025-04-07   5.8400   5.9255   5.7994     5.8400
